### Test case LVV-T153: Verify implementation of Provide Engineering and Facility Database Archive

Demonstrate Engineering and Facilities Data (images, associated metadata, and observatory environment and control data) are archived and available for public access within **L1PublicT (24 hours)**.

#### Discussion:

This can be verified using the "ConsDB" (Consolidated database), which is a transformed version of telemetry from the Engineering Facilities Database (EFD), with additional tables and their columns populated by quantities derived from images immediately after they are obtained.

In particular, this test will use on-sky data and telemetry from LATISS observing in February 2025. All queries will be limited to the current incoming data stream, and will be executed while LATISS observing is actively being conducted.

#### Look up the current time and print it to the screen:

In [1]:
import datetime

# Note that Rubin Observatory reports quantities in TAI, but for
#   our present purposes, the 37-second difference between UTC and TAI
#   is unimportant.
time_now = datetime.datetime.now(datetime.UTC)
print(time_now)

2025-02-28 03:22:38.054606+00:00


The current UTC time is February 28, 2025, so we will select an observing date of Feb. 27, 2025 (the day_obs is the local date in Chile on which the observing night began).

In [2]:
# Parameters to select a single observation date and the LATISS instrument:
day_obs = "2025-02-27"
instrument = "latiss"

Import packages, and set some variables for authentication to ConsDB:

In [3]:
import os
import numpy as np
import pandas as pd
import requests
from IPython.display import Markdown, display, display_markdown
from lsst.summit.utils import ConsDbClient
from astropy.table import Table

URL = "http://consdb-pq.consdb:8080/consdb"

os.environ["no_proxy"] += ",.consdb"

access_token = os.getenv("ACCESS_TOKEN")
headers = {"Authorization": f"Bearer {access_token}"}

%matplotlib inline

Authenticate to and access the ConsDB:

In [4]:
client = ConsDbClient(URL)
print(client)

display_markdown("### Consolidated Database is accessible", raw=True)

### Consolidated Database is accessible

Query the ConsDB over a range of a few days

In [5]:
day_obs_int = int(day_obs.replace("-", ""))
print(f'Date: {day_obs_int}')

# Query the visit table
visit_query1 = f"""
    SELECT * FROM cdb_{instrument}.visit1
    where day_obs <= {day_obs_int} AND day_obs > {day_obs_int-1}
"""

# Print some schema information
try:
    print('\nlist of instruments in client.schema:')
    print(client.schema())  # list the instruments
    print('\nTables in client.schema("latiss"):')
    print(client.schema(f"{instrument}"))  # list tables for an instrument
except requests.HTTPError or requests.JSONDecodeError:
    print(client.schema())  # list the instruments
    print(client.schema(f"{instrument}"))  # list tables for an instrument

# Execute the query:
try:
    visits_comcam = client.query(visit_query1).to_pandas()
except requests.HTTPError or requests.JSONDecodeError:
    # Try twice
    visits_comcam = client.query(visit_query1).to_pandas()

# Print some stats to the screen:
if len(visits_comcam) > 0:
    print(f"\nRetrieved {len(visits_comcam)} visits from consdb")
    obj_vis = len(visits_comcam.query('img_type == "OBJECT"'))
    print(f"{obj_vis} of these are object images")


Date: 20250227

list of instruments in client.schema:
['latiss', 'lsstcam', 'lsstcamsim', 'lsstcomcam', 'lsstcomcamsim', 'startrackerfast', 'startrackernarrow', 'startrackerwide']

Tables in client.schema("latiss"):
['cdb_latiss.exposure', 'cdb_latiss.exposure_flexdata', 'cdb_latiss.exposure_flexdata_schema', 'cdb_latiss.visit1_quicklook', 'cdb_latiss.ccdexposure', 'cdb_latiss.ccdexposure_camera', 'cdb_latiss.ccdexposure_flexdata', 'cdb_latiss.ccdexposure_flexdata_schema', 'cdb_latiss.exposure_quicklook', 'cdb_latiss.ccdvisit1_quicklook']

Retrieved 319 visits from consdb
272 of these are object images


#### Print the list of columns in this table:

In [6]:
cols = [col for col in visits_comcam.columns]
print(cols)

['visit_id', 'exposure_name', 'controller', 'day_obs', 'seq_num', 'physical_filter', 'band', 's_ra', 's_dec', 'sky_rotation', 'azimuth_start', 'azimuth_end', 'azimuth', 'altitude_start', 'altitude_end', 'altitude', 'zenith_distance_start', 'zenith_distance_end', 'zenith_distance', 'airmass', 'exp_midpt', 'exp_midpt_mjd', 'obs_start', 'obs_start_mjd', 'obs_end', 'obs_end_mjd', 'exp_time', 'shut_time', 'dark_time', 'group_id', 'cur_index', 'max_index', 'img_type', 'emulated', 'science_program', 'observation_reason', 'target_name', 'air_temp', 'pressure', 'humidity', 'wind_speed', 'wind_dir', 'dimm_seeing', 'shut_lower', 'shut_upper', 'focus_z', 'dome_azimuth', 'simulated', 's_region']


#### Display statistics related to the date/time and duration of the observation, and the filter:

Note that this is spectroscopic observing, so the filter and band are mostly not relevant, other than as order-blocking filters.

In [7]:
obs_columns = ['day_obs', 'obs_start', 'obs_end', 'exp_time', 'shut_time',
               'dark_time', 'physical_filter', 'band']

obs_dict = {}

for col in obs_columns:
    obs_dict[col] = visits_comcam.loc[:15][col].values

tab = Table(obs_dict)
tab

day_obs,obs_start,obs_end,exp_time,shut_time,dark_time,physical_filter,band
int64,object,object,float64,float64,float64,object,object
20250227,2025-02-27T18:47:40.160000,2025-02-27T18:47:40.170000,0.0,0.0,0.010358572006225586,empty~empty,EMPTY
20250227,2025-02-27T18:47:45.949000,2025-02-27T18:47:48.175000,2.0,2.0,2.2258059978485107,empty~empty,EMPTY
20250227,2025-02-27T18:59:17.087000,2025-02-27T18:59:19.314000,2.0,2.0,2.2272896766662598,empty~empty,EMPTY
20250227,2025-02-27T19:02:43.612000,2025-02-27T19:02:45.832000,2.0,2.0,2.2208499908447266,BG40_65mm_1~blue300lpmm_qn1,g
20250227,2025-02-27T23:54:21.787000,2025-02-27T23:54:23.010000,1.0,1.0,1.2234866619110107,SDSSr_65mm~empty,r
20250227,2025-02-27T23:54:53.035000,2025-02-27T23:54:54.272000,1.0,1.0,1.236938238143921,SDSSr_65mm~empty,r
20250227,2025-02-27T23:55:21.684000,2025-02-27T23:55:22.906000,1.0,1.0,1.222588062286377,SDSSr_65mm~empty,r
20250227,2025-02-28T00:03:33.470000,2025-02-28T00:03:34.701000,1.0,1.0,1.2312712669372559,empty~empty,EMPTY
20250227,2025-02-28T00:06:30.832000,2025-02-28T00:06:30.841000,0.0,0.0,0.008863449096679688,empty~empty,EMPTY


Notice that the times printed (in "obs_start") are much less than 24 hours before the current time shown above. We have thus demonstrated that the databases are populated within L1PublicT = 24 hours.

#### Display quantities related the dome and telescope pointing and orientation:

In [8]:
tel_columns = ['sky_rotation', 'azimuth_start', 'azimuth_end', 'azimuth',
               'altitude_start', 'altitude_end', 'altitude']

tel_dict = {}

for col in tel_columns:
    tel_dict[col] = visits_comcam.loc[10:15][col].values

tab = Table(tel_dict)
tab

sky_rotation,azimuth_start,azimuth_end,azimuth,altitude_start,altitude_end,altitude
float64,float64,float64,float64,float64,float64,float64
168.3219184381463,62.563592371483466,62.54056778117291,62.55208007632819,24.379946331256022,24.367116947327034,24.373531639291528
161.16652473660974,56.139836305170576,55.90782409026456,56.02383019771757,22.322240357050475,22.223523837588544,22.27288209731951
161.1664690224672,55.80183121577328,55.56731106630731,55.684571141040294,22.178683972314246,22.08032958241469,22.12950677736447
161.16632993313456,55.251723424793774,55.01308005935227,55.13240174207302,21.949513501642514,21.851972921574585,21.90074321160855
161.1664337871126,54.91107898872813,54.66976748145442,54.790423235091275,21.810476086022106,21.713239660894345,21.761857873458226
161.16641643551347,54.43380073448416,54.37455154169263,54.404176138088395,21.619196761374013,21.595679437209597,21.607438099291805


#### Display quantities related to the airmass, zenith distance, and environmental conditions of observations:

In [9]:
env_columns1 = ['zenith_distance_start', 'zenith_distance_end', 'zenith_distance', 'airmass']

env_dict1 = {}

for col in env_columns1:
    env_dict1[col] = visits_comcam.loc[10:15][col].values

tab = Table(env_dict1)
tab

zenith_distance_start,zenith_distance_end,zenith_distance,airmass
float64,float64,float64,object
65.62005366874398,65.63288305267297,65.62646836070847,1.0972570877462795
67.67775964294952,67.77647616241146,67.72711790268049,1.0804418787479548
67.82131602768575,67.91967041758531,67.87049322263553,1.079330094369854
68.05048649835749,68.14802707842541,68.09925678839144,1.0775856427851642
68.1895239139779,68.28676033910565,68.23814212654177,1.076533996842899
68.38080323862599,68.4043205627904,68.39256190070819,1.0751056583289715


In [10]:
env_columns2 = ['air_temp', 'pressure', 'humidity', 'wind_speed', 'wind_dir', 'dimm_seeing', 'focus_z']

env_dict2 = {}

for col in env_columns2:
    env_dict2[col] = visits_comcam.loc[10:15][col].values

tab = Table(env_dict2)
tab

air_temp,pressure,humidity,wind_speed,wind_dir,dimm_seeing,focus_z
float64,float64,float64,float64,float64,object,float64
14.399999618530273,74345.0,58.0,2.9189999103546143,11.944999694824219,None,0.0
14.274999618530273,74360.0,61.13750076293945,2.418600082397461,19.364999771118164,None,0.800000011920929
14.149999618530273,74360.0,61.275001525878906,1.9182000160217285,11.835000038146973,None,-0.801099956035614
13.975000381469727,74360.0,61.25,2.251800060272217,5.079999923706055,None,0.7515544891357422
13.975000381469727,74360.0,62.07500076293945,2.0016000270843506,5.98527717590332,None,-0.8495454788208008
13.949999809265137,74360.0,62.525001525878906,1.9182000160217285,1.3399637937545776,None,-0.04824773594737053


In [11]:
# Query the ccdexposure table
ccdexp_query = f"""
    SELECT * FROM cdb_{instrument}.ccdexposure
    where day_obs <= {day_obs_int} AND day_obs > {day_obs_int-3}
"""

# Execute the query:
try:
    ccdexp_comcam = client.query(ccdexp_query).to_pandas()
except requests.HTTPError or requests.JSONDecodeError:
    # Try twice
    ccdexp_comcam = client.query(ccdexp_query).to_pandas()


In [12]:
ccdexp_comcam[-5:]

,ccdexposure_id,exposure_id,detector,s_region,day_obs,seq_num
1664,46439414528,2025022700315,0,Polygon ICRS 193.389047 -18.547848 193.285059 ...,20250227,315
1665,46439414784,2025022700316,0,None,20250227,316
1666,46439415040,2025022700317,0,None,20250227,317
1667,46439415296,2025022700318,0,Polygon ICRS 193.389028 -18.547898 193.285008 ...,20250227,318
1668,46439415552,2025022700319,0,Polygon ICRS 193.389028 -18.547898 193.285008 ...,20250227,319


## Results

We see that the ConsDB contains information taken directly from image headers, transformed data from the EFD, and derived data based on image processing. As required, these data include information about each exposure, include the telescope and instrument configuration, telemetry from the telescope, environmental and pointing information, and details about the camera.

We have demonstrated that the ConsDB tables were populated in less than L1PublicT=24 hours, as required.

The result of this test is **PASS**.